In [49]:
# Use np.nan for NaN values, do not import NaN directly from numpy

from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date, auto_adjust=False).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
['VLTO', 'SOLV', 'SW', 'GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-09-29 00:00:00 -> 2023-09-27) (Yahoo error = "Data doesn\'t exist for startDate = 1443499200, endDate = 1695787200")')


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.251009   33.740002   34.060001   33.240002   33.360001   
           AAPL     24.536383   27.264999   28.377501   26.965000   28.207500   
           ABBV     35.061214   52.790001   54.189999   51.880001   53.099998   
           ABT      32.820744   39.500000   40.150002   39.029999   39.259998   
           ACGL     23.217773   24.416668   24.456667   24.100000   24.170000   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      87.701065   89.519997   90.849998   89.500000   90.379997   
           YUM     119.860718  124.010002  124.739998  123.449997  124.239998   
           ZBH     110.800156  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     173.604706  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
           ACGL      1888800.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[980418 rows x 6 columns]

In [50]:

# Compute RSI manually
def compute_rsi(series, length=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=length, min_periods=length).mean()
    avg_loss = loss.rolling(window=length, min_periods=length).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Compute Bollinger Bands manually
def compute_bbands(series, length=20, num_std=2):
    sma = series.rolling(window=length, min_periods=length).mean()
    std = series.rolling(window=length, min_periods=length).std()
    upper_band = sma + num_std * std
    lower_band = sma - num_std * std
    return lower_band, sma, upper_band

# Compute Bollinger Bands (on log1p of adj close)
def bbands_transform(x):
    log_prices = np.log1p(x)
    low, mid, high = compute_bbands(log_prices, length=20)
    return pd.DataFrame({
        'bb_low': low,
        'bb_mid': mid,
        'bb_high': high
    }, index=x.index)

# Compute ATR manually and standardize it
def compute_atr(stock_data, length=14):
    high = stock_data['high']
    low = stock_data['low']
    close = stock_data['adj close']

    prev_close = close.shift(1)
    tr = pd.concat([
        high - low,
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    atr = tr.rolling(window=length, min_periods=length).mean()
    atr_zscore = (atr - atr.mean()) / atr.std()
    return atr_zscore

# Compute MACD manually and standardize it
def compute_macd(series, fast=12, slow=26):
    ema_fast = series.ewm(span=fast, min_periods=fast).mean()
    ema_slow = series.ewm(span=slow, min_periods=slow).mean()
    macd_line = ema_fast - ema_slow
    macd_zscore = (macd_line - macd_line.mean()) / macd_line.std()
    return macd_zscore

# Garman-Klass volatility estimator
df['garman_klass_vol'] = (
    ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2
    - (2 * np.log(2) - 1) * ((np.log(df['adj close']) - np.log(df['open'])) ** 2)
)

# Compute RSI grouped by level 1
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: compute_rsi(x, length=20))

# Compute Bollinger Bands grouped by level 1
bb_df = df.groupby(level=1)['adj close'].apply(bbands_transform)
df[['bb_low', 'bb_mid', 'bb_high']] = bb_df.reset_index(level=0, drop=True)

# Compute ATR grouped by level 1
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# Compute MACD grouped by level 1
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].transform(lambda x: compute_macd(x))

df['dollar_volume'] = (df['adj close']* df['volume'])/1e6

df


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.251009   33.740002   34.060001   33.240002   33.360001   
           AAPL     24.536383   27.264999   28.377501   26.965000   28.207500   
           ABBV     35.061214   52.790001   54.189999   51.880001   53.099998   
           ABT      32.820744   39.500000   40.150002   39.029999   39.259998   
           ACGL     23.217773   24.416668   24.456667   24.100000   24.170000   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      87.701065   89.519997   90.849998   89.500000   90.379997   
           YUM     119.860718  124.010002  124.739998  123.449997  124.239998   
           ZBH     110.800156  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     173.604706  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.001351        NaN       NaN   
           AAPL    293461600.0         -0.006207        NaN       NaN   
           ABBV     12842800.0         -0.065607        NaN       NaN   
           ABT      12287500.0         -0.011997        NaN       NaN   
           ACGL      1888800.0         -0.000516        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000238  22.653326  4.472030   
           YUM       1500600.0         -0.000443  36.971476  4.791669   
           ZBH       3610500.0         -0.000229  41.303613  4.738303   
           ZBRA       355400.0          0.000133  21.657597  5.397402   
           ZTS       1463200.0         -0.000036  37.270293  5.143515   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      70.389773  
           AAPL         NaN       NaN       NaN       NaN    7200.486118  
           ABBV         NaN       NaN       NaN       NaN     450.284165  
           ABT          NaN       NaN       NaN       NaN     403.284887  
           ACGL         NaN       NaN       NaN       NaN      43.853730  
...                     ...       ...       ...       ...            ...  
2023-09-26 XYL     4.556072  4.640113 -3.011218 -2.157409     115.975888  
           YUM     4.822408  4.853147 -2.828854 -1.367168     179.862993  
           ZBH     4.778997  4.819692 -2.199153 -0.878964     400.043962  
           ZBRA    5.539167  5.680932 -0.078248 -1.600810      79.595386  
           ZTS     5.203852  5.264190 -0.506075 -1.186640     254.018406  

[980418 rows x 14 columns]

In [51]:
# Unstack the DataFrame to get a multi-index DataFrame with tickers as columns
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close', 'close']]
# Unstack the DataFrame to get a multi-index DataFrame with tickers as columns

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
            df.unstack()[last_cols].resample('M').last().stack('ticker')], axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2015-11-30 A          134.988347   38.734955         -0.002430  70.422525   
           AAPL      4005.252599   26.729130         -0.003654  48.630908   
           ABBV       325.730917   38.977573         -0.070930  46.106796   
           ABT        207.499329   37.540981         -0.013992  50.738837   
           ACGL        28.174423   22.970539         -0.001121  30.793654   
...                          ...         ...               ...        ...   
2023-09-30 EXE        116.689768   79.277847         -0.000348  43.297780   
           COIN       506.793576   70.519997          0.001007  46.416531   
           CEG        195.364207  107.145683         -0.000064  56.041841   
           GEHC       211.929506   66.022324          0.000183  41.379362   
           KVUE       646.494220   19.281166         -0.001506  32.942917   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2015-11-30 A       3.536910  3.611226  3.685542 -0.915931  0.440817  
           AAPL    3.276064  3.320493  3.364922 -0.494416 -0.209164  
           ABBV    3.690632  3.740093  3.789553  0.344526  0.018850  
           ABT     3.636157  3.658567  3.680977  0.049596  0.121016  
           ACGL    3.177527  3.195190  3.212853 -1.119424 -0.551905  
...                     ...       ...       ...       ...       ...  
2023-09-30 EXE     4.368069  4.422967  4.477865 -1.840200 -0.828974  
           COIN    4.270839  4.378785  4.486731 -1.113593  0.008323  
           CEG     4.644504  4.685718  4.726932 -0.095326  0.371211  
           GEHC    4.152334  4.211363  4.270392 -0.700162 -1.176709  
           KVUE    2.977257  3.054020  3.130783 -2.231693 -1.514010  

[46290 rows x 9 columns]

In [52]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))


data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data


adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2016-10-31 AAPL     26.090458         -0.002767  53.857832  3.288994   
           ABBV     38.834366         -0.056807  25.623947  3.717208   
           ABT      33.619495         -0.009785  33.585028  3.534045   
           ACN     101.760162         -0.006263  38.045346  4.619587   
           ADBE    107.510002          0.000059  46.160151  4.679120   
...                       ...               ...        ...       ...   
2023-09-30 CRWD    160.479996          0.000144  65.687578  5.024174   
           PLTR     13.960000          0.000214  45.277776  2.699917   
           DASH     74.580002          0.000326  40.373286  4.327250   
           ABNB    132.279999          0.000213  56.841285  4.854868   
           COIN     70.519997          0.001007  46.416531  4.270839   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2016-10-31 AAPL    3.318620  3.348245 -1.004341 -0.194256  
           ABBV    3.772733  3.828259 -0.101567 -0.758457  
           ABT     3.585802  3.637559 -1.060312 -0.646878  
           ACN     4.631524  4.643462 -0.491808 -0.132574  
           ADBE    4.694639  4.710159 -1.180271 -0.107781  
...                     ...       ...       ...       ...  
2023-09-30 CRWD    5.103696  5.183218 -0.894312  0.250658  
           PLTR    2.779743  2.859570 -0.490510 -0.433592  
           DASH    4.403906  4.480561 -1.083109 -0.102635  
           ABNB    4.940924  5.026980 -0.961350 -0.010573  
           COIN    4.378785  4.486731 -1.113593  0.008323  

[12516 rows x 8 columns]

In [ ]:
# Calculate returns for each ticker in the DataFrame
# This function calculates the returns for each ticker based on the adjusted close prices
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1,2, 3, 6, 9, 12]

    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close'].pct_change(lag).pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff), 
                                                                            upper=x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1))
    return df
    

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data 

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2018-10-31 AAPL     51.952988         -0.000551  41.783782  3.932914   
           ABBV     58.368000         -0.040254  24.322682  4.070619   
           ABT      61.543369         -0.004473  41.791091  4.093104   
           ACN     143.307495         -0.002640  35.720103  4.915307   
           ADBE    245.759995          0.000288  41.700753  5.451019   
...                       ...               ...        ...       ...   
2023-09-30 WMT      53.052731         -0.000186  60.912189  3.974496   
           XOM     109.552101         -0.001011  65.920112  4.652095   
           MRNA     98.120003          0.000146  32.702048  4.579843   
           UBER     44.270000          0.000441  50.373131  3.805210   
           CRWD    160.479996          0.000144  65.687578  5.024174   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2018-10-31 AAPL    3.972273  4.011632 -0.044754 -0.446072  -0.030478   
           ABBV    4.195407  4.320195  0.761580 -1.979003  -0.168082   
           ABT     4.136193  4.179282  0.342480 -0.510998  -0.056387   
           ACN     4.981695  5.048084 -0.171907 -1.022769  -0.065449   
           ADBE    5.520491  5.589962  0.739337 -0.755933  -0.089609   
...                     ...       ...       ...       ...        ...   
2023-09-30 WMT     3.992416  4.010337 -2.611573  0.394852  -0.000677   
           XOM     4.693220  4.734345 -1.141950  1.408626   0.046947   
           MRNA    4.685332  4.790820 -0.484476 -0.377001  -0.132219   
           UBER    3.862227  3.919244 -0.642241 -0.128020  -0.062672   
           CRWD    5.103696  5.183218 -0.894312  0.250658  -0.015641   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2018-10-31 AAPL    -0.019453   0.048955   0.049216   0.031518    0.023037  
           ABBV    -0.090324  -0.051576  -0.031893  -0.036005   -0.009087  
           ABT      0.017674   0.018394   0.030307   0.013075    0.021747  
           ACN     -0.030053  -0.000564   0.008483  -0.000163    0.010026  
           ADBE    -0.034267   0.001469   0.017396   0.023294    0.028623  
...                      ...        ...        ...        ...         ...  
2023-09-30 WMT      0.010014   0.012354   0.017574   0.016553    0.020256  
           XOM      0.046139   0.030496   0.012838   0.008747    0.027037  
           MRNA    -0.086803  -0.068763  -0.071952  -0.064976   -0.015431  
           UBER    -0.053920   0.008422   0.057244   0.066838    0.043691  
           CRWD    -0.003656   0.029981   0.026391   0.046986   -0.002216  

[8285 rows x 14 columns]

In [ ]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.names = ['date']

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

,Mkt-RF,SMB,HML,RMW,CMA,return_1m
date,,,,,,
2018-10-31,-0.0765,-0.0438,0.0337,0.0100,0.0365,-0.030478
2018-11-30,0.0171,-0.0085,0.0039,-0.0063,0.0044,-0.161253
2018-12-31,-0.0955,-0.0283,-0.0194,-0.0003,0.0026,-0.116698
2019-01-31,0.0837,0.0290,-0.0039,-0.0070,-0.0168,0.055154
2019-02-28,0.0342,0.0175,-0.0267,0.0016,-0.0156,0.044776
2019-03-31,0.0110,-0.0353,-0.0416,0.0091,-0.0089,0.097026
2019-04-30,0.0397,-0.0111,0.0215,0.0157,-0.0219,0.056436
2019-05-31,-0.0692,-0.0150,-0.0243,-0.0051,0.0173,-0.124213
2019-06-30,0.0692,0.0037,-0.0067,0.0091,-0.0040,0.130519
